In [1]:
import numpy as np

## III. View和copy

### numpy array数据结构
 - numpy array的数据结构中包括两部分内容，一是contiguous data buffer，这里连续存储着data elements，另一部分是metadata，metadata中记录了data type，strides(seperation for dims)和其他帮助呈现ndarray形态的信息等。
 - 不改变data buffer，仅仅改变metadata中的部分信息就可以形成新的ndarray，这些新的ndarray是databuffer的一个View，因为它只改变了buffer中数据的呈现形式。
 - 这种设计使得ndarray的运算效率很高，因为不用频繁复制原始数据。

### view
1. 如果只通过改变原ndarray的metadata信息中的offsets和strides就能得到目标new array的话，会创建View。不然的话就会做copy。
2. 改变view中元素的值同时也会改变base array中的data buffer
3. 常见的创建view的场景:通常indexing、slicing和reshape得到的都是View
4. 可以用base attribute的值来判断是View还是copy, 如果是View，那么A.base会输出真实的data。如果是copy，输出为None
5. 但是要注意View是新建了metadata，要区别于python里面新建reference。\
(1) 一般赋值没有新建view，也不发生copy，只是将新的varible reference到已经存在的array上 \
(2) ndarray做为参数传递给函数的时候，得到的也是View

In [2]:
x = np.arange(6)
y = x[1:3]           # slicing
z = x.reshape(2, 3)  # reshape
y.base, z.base

(array([0, 1, 2, 3, 4, 5]), array([0, 1, 2, 3, 4, 5]))

In [3]:
# 变量赋值
a = np.array([[ 0,  1,  2,  3],
              [ 4,  5,  6,  7],
              [ 8,  9, 10, 11]])
b = a            # no new object is created
b.base is None         # a and b are two names for the same ndarray object

True

In [4]:
# 函数传参
def f(x):
    print(id(x))

id(a), f(a) # id is a unique identifier of an object 

131069992141424


(131069992141424, None)

### copy
1. 如果一般的indexing、slicing和reshape等如果要copy的话，直接用A.copy()
2. advanced indexing会直接得到copy

#### 发生copy的reshape
 - 大多数情况下，只要改变stride(dim之间的分界距离)就能为reshape创建新的View，但是有时候reshape无法通过简单改变stride来实现。
 - 比如，array如果先做了transpose，再做reshape，新的array的元素位置在原始array的data buffer里面可能就不连续了。这时候会自动做copy
 - 注意，发生copy的时候，不见得新生成的array就是base

In [5]:
a = np.arange(12)
b = a.reshape(4, 3)
b.base is a, b, b.ravel()

(True,
 array([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]))

In [6]:
c = b.T
c.base is a, c, c.ravel()

(True,
 array([[ 0,  3,  6,  9],
        [ 1,  4,  7, 10],
        [ 2,  5,  8, 11]]),
 array([ 0,  3,  6,  9,  1,  4,  7, 10,  2,  5,  8, 11]))

In [7]:
d = c.reshape(3, 2, 2)
d.base is a, d, d.ravel()

(True,
 array([[[ 0,  3],
         [ 6,  9]],
 
        [[ 1,  4],
         [ 7, 10]],
 
        [[ 2,  5],
         [ 8, 11]]]),
 array([ 0,  3,  6,  9,  1,  4,  7, 10,  2,  5,  8, 11]))

In [8]:
e = c.reshape(2, 3, 2)       # e发生了copy，但是e自己本身并不是base
e.base is a, e.base is None, e, e.base, e.ravel()

(False,
 False,
 array([[[ 0,  3],
         [ 6,  9],
         [ 1,  4]],
 
        [[ 7, 10],
         [ 2,  5],
         [ 8, 11]]]),
 array([[ 0,  3,  6,  9],
        [ 1,  4,  7, 10],
        [ 2,  5,  8, 11]]),
 array([ 0,  3,  6,  9,  1,  4,  7, 10,  2,  5,  8, 11]))

In [9]:
e[0, 0, 0] = 100
e, a

(array([[[100,   3],
         [  6,   9],
         [  1,   4]],
 
        [[  7,  10],
         [  2,   5],
         [  8,  11]]]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]))

In [10]:
f = e.T        # f只是View，f的base不是e, 但f和e有共同的base
f.base is e, f.base is e.base, f

(False,
 True,
 array([[[100,   7],
         [  6,   2],
         [  1,   8]],
 
        [[  3,  10],
         [  9,   5],
         [  4,  11]]]))

In [11]:
f[0, 0, 1] = 99  # 改变f中的元素，也会改变e中对应位置元素的值 
f, e

(array([[[100,  99],
         [  6,   2],
         [  1,   8]],
 
        [[  3,  10],
         [  9,   5],
         [  4,  11]]]),
 array([[[100,   3],
         [  6,   9],
         [  1,   4]],
 
        [[ 99,  10],
         [  2,   5],
         [  8,  11]]]))